# Trial Code

### Notebook forked from https://www.kaggle.com/hiteshkumars/mmdetection-for-segmentation-inference

In [2]:
import pandas as pd
import numpy as np
import os
import cv2
from tqdm import tqdm #adds progress bar
import pickle
from itertools import groupby
from pycocotools import mask as mutils
from pycocotools import _mask as coco_mask
import matplotlib.pyplot as plt
import os
import base64 #converts binary to ASCII characters
import typing as t #annotates functions like linter
import zlib #compresses files
import random
random.seed(0)

In [3]:
exp_name = "v4"
conf_name = "mask_rcnn_s101_fpn_syncbn-backbone+head_mstrain_1x_coco"
model_name = 'mask_rcnn_resnest101_v5_ep9'
ROOT = '/home/chad/GitHub/Protein_Atlas/'
train_or_test = 'test'
df = pd.read_csv(os.path.join(ROOT, 'input/sample_submission.csv'))
if len(df) == 559:
    debug = True
    df = df[:3] #Subset rows 0 - 2
else:
    debug = False

In [4]:
print(df)

                                     ID  ImageWidth  ImageHeight  \
0  0040581b-f1f2-4fbe-b043-b6bfea5404bb        2048         2048   
1  004a270d-34a2-4d60-bbe4-365fca868193        2048         2048   
2  00537262-883c-4b37-a3a1-a4931b6faea5        2048         2048   

           PredictionString  
0  0 1 eNoLCAgIMAEABJkBdQ==  
1  0 1 eNoLCAgIMAEABJkBdQ==  
2  0 1 eNoLCAgIMAEABJkBdQ==  


## Helper Functions

In [5]:
#The '->' at the end of the function is function annotation. 
#The typing package interprets the meaning of the function

#The same goes for the input. It is just saying it takes a 'mask' that is an np.ndarray.
#t.Text is the output of the function. 

#COCO is a large-scale object detection, segmentation, and captioning dataset. Its API
# has masking tools. https://cocodataset.org/#home

def encode_binary_mask(mask: np.ndarray) -> t.Text:
  """Converts a binary mask into OID (OpenImage?) challenge encoding ascii text."""

  # check input mask --
  if mask.dtype != np.bool:
    raise ValueError(
        "encode_binary_mask expects a binary mask, received dtype == %s" %
        mask.dtype)

  mask = np.squeeze(mask)
  if len(mask.shape) != 2:
    raise ValueError(
        "encode_binary_mask expects a 2d mask, received shape == %s" %
        mask.shape)

  # convert input mask to expected COCO API input --
  mask_to_encode = mask.reshape(mask.shape[0], mask.shape[1], 1)
  mask_to_encode = mask_to_encode.astype(np.uint8)
  mask_to_encode = np.asfortranarray(mask_to_encode)

  # RLE encode mask --
  encoded_mask = coco_mask.encode(mask_to_encode)[0]["counts"]

  # compress and base64 encoding -- Encodes binary data to printable ASCII characters
  binary_str = zlib.compress(encoded_mask, zlib.Z_BEST_COMPRESSION)
  base64_str = base64.b64encode(binary_str)
  return base64_str.decode()

![title](img/typing_text.png)

In [6]:
#Assert raises AssertionError is value is False. Used for debugging

#cv2.IMREAD_UNCHANGED returns image as-is with alpha channel

#Change image to 'uint8' to keep values in 0-255 range.

def read_img(image_id, color, train_or_test='train', image_size=None):
    filename = f'{ROOT}/input/{train_or_test}/{image_id}_{color}.png'
    assert os.path.exists(filename), f'not found {filename}'
    img = cv2.imread(filename, cv2.IMREAD_UNCHANGED)
    if image_size is not None:
        img = cv2.resize(img, (image_size, image_size))
    if img.dtype == 'uint16':
        img = (img/256).astype('uint8') 
    return img

In [7]:
#I believe the (1,2,0) for the transpose function is the order. 
# It might mean change to green, blue, red but not sure why yet. 

def load_RGBY_image(image_id, train_or_test='train', image_size=None):
    red = read_img(image_id, "red", train_or_test, image_size)
    green = read_img(image_id, "green", train_or_test, image_size)
    blue = read_img(image_id, "blue", train_or_test, image_size)
    # using rgb only here
    #yellow = read_img(image_id, "yellow", train_or_test, image_size)
    stacked_images = np.transpose(np.array([red, green, blue]), (1,2,0))
    return stacked_images

In [8]:
def print_masked_img(image_id, mask):
    img = load_RGBY_image(image_id, train_or_test)
    
    plt.figure(figsize=(15, 15))
    plt.subplot(1, 3, 1)
    plt.imshow(img)
    plt.title('Image')
    plt.axis('off')
    
    plt.subplot(1, 3, 2)
    plt.imshow(mask)
    plt.title('Mask')
    plt.axis('off')
    
    plt.subplot(1, 3, 3)
    plt.imshow(img)
    plt.imshow(mask, alpha=0.6)
    plt.title('Image + Mask')
    plt.axis('off')
    plt.show()

## Generate files for mmdetection

MMDetection is an open source object detection toolbox based on PyTorch. It is a part of the OpenMMLab project developed by Multimedia Laboratory, CUHK.

![title](img/coco_test_12510.jpg)

In [11]:
out_image_dir = f'{ROOT}/output/mmdet_{exp_name}_{train_or_test}/'
!mkdir -p {out_image_dir}

annos = []
for idx in tqdm(range(len(df))):
    image_id = df.iloc[idx].ID
    img = load_RGBY_image(image_id, train_or_test)
    
    cv2.imwrite(f'{out_image_dir}/{image_id}.jpg', img)
    ann = {
        'filename': image_id+'.jpg',
        'width': img.shape[1],
        'height': img.shape[0],
        'ann': {
            'bboxes': None,
            'labels': None,
            'masks': None
        }
    }
    annos.append(ann)
    
with open(f'{ROOT}pickle_tst.pkl', 'wb') as f:
    pickle.dump(annos, f)

100%|██████████| 3/3 [00:00<00:00,  3.98it/s]
